In [ ]:
import os
import struct
import pickle
from collections import OrderedDict


class Segment:
    """
    A log-structured segment for appending key-value data.
    """
    def __init__(self, path, segment_id):
        self.path = os.path.join(path, f"segment_{segment_id}.log")
        self.file = open(self.path, "ab+")
        self.id = segment_id

    def write(self, key, value):
        offset = self.file.tell()
        serialized = pickle.dumps((key, value))
        self.file.write(struct.pack("I", len(serialized)))
        self.file.write(serialized)
        self.file.flush()
        return offset

    def read(self, offset):
        self.file.seek(offset)
        size = struct.unpack("I", self.file.read(4))[0]
        serialized = self.file.read(size)
        return pickle.loads(serialized)

    def close(self):
        self.file.close()

class SegmentManager:
    """
    Manages multiple log-structured segments.
    """
    def __init__(self, path):
        self.path = path
        self.segments = OrderedDict()
        self.counter = 0

    def new_segment(self):
        segment = Segment(self.path, self.counter)
        self.segments[self.counter] = segment
        self.counter += 1
        return segment

    def close(self):
        for segment in self.segments.values():
            segment.close()